# Architecture

## High Level

<img src="./helpers/wal-replication.png" alt="drawing"/>

## Primary

**File Based**

The primary is just doing continuous WAL archiving in an accessible to standby machine place.

## Standby Mode

A server enters standby mode (standby loop) if a `standby.signal` file exists in the data directory when the server is started.

Standby mode is exited and the server switches to normal operation when `pg_ctl promote` is run, or `pg_promote()` is called.

<img src="./helpers/Replication - Standby Loop.png" alt="drawing"/>

## Streaming

<img src="./helpers/streaming-replication-architecture.png" alt="drawing" width="700"/>

1. Start the primary and standby servers.
1. The standby server starts the startup process.
1. The standby server starts a walreceiver process.
1. The walreceiver sends a connection request to the primary server. If the primary server is not running, the walreceiver sends these requests periodically.
1. When the primary server receives a connection request, it starts a walsender process and a TCP connection is established between the walsender and walreceiver.
1. Handshake: the walreceiver sends the latest LSN (Log Sequence Number) of standby’s database cluster.
1. If the standby’s latest LSN is less than the primary’s latest LSN (Standby’s LSN Primary’s LSN), the walsender sends WAL data from the former LSN to the latter LSN. These WAL data are provided by WAL segments stored in the primary’s pg_wal subdirectory. The standby server then replays the received WAL data. In this phase, the standby catches up with the primary, so it is called catch-up.
1. Streaming Replication begins to work.

## Cascading Replication

Idea: use a standby as an upstream of data changes in the primary to not have to connect directly to the primary

<img src="./helpers/cascading-replication.png" alt="drawing" height="500"/>